In [ ]:
using Pkg
Pkg.activate(".")

# Methods

So far, we defined all functions (with some exceptions) without annotating the types of input arguments. When the type annotation is omitted, the default behaviour in Julia is to allow values to be of any type. One can write many useful functions without stating the types. When additional expressiveness is needed, it is easy to introduce type annotations into previously *untyped* code.

In Julia, one function consists of multiple methods. A prime example is the `convert` function. When a user calls a function, the process of choosing which method to execute is called dispatch. The dispatch system in Julia decides which method to execute based on:

- the number of function arguments;
- the types of function arguments.

Using all function arguments to choose which method should be invoked is known as **multiple dispatch**.

As an example of multiple dispatch, we define the `product` function that computes the product of two numbers.

In [ ]:
product(x, y) = x * y

In the REPL, we can see that the `product` function has only one method. In this case, we defined the method for any two input arguments without type specification.

In [ ]:
product(1, 4.5)
product(2.4, 3.1)

The `methods` function lists all methods for a function.

In [ ]:
methods(product)

Because we did not specify types of input arguments, the `product` function accepts arguments of all types. For some inputs, such as symbols, the `*` operator will not work.

In [ ]:
product(:a, :b)

We can avoid such errors by specifying types of input arguments. Since we want to create a function that computes the product of two numbers, it makes sense to allow input arguments to be only numbers.

In [ ]:
product(x::Number, y::Number) = x * y
product(x, y) = throw(ArgumentError("product is defined for numbers only."))

In [ ]:
methods(product)

Now, we have a function with two methods, that returns a product if the input arguments are numbers, and throws an error otherwise.

In [ ]:
product(1, 4.5)

In [ ]:
product(:a, :b)

In [ ]:
product("a", "b")

## Type hierarchy

It is always better to use abstract types like `Number` or `Real` instead of concrete types like `Float64`, `Float32`, or `Int64`. The reason is that if we use an abstract type, the function will work for all its subtypes. To find a supertype for a specific type, we can use the `supertype` function from the `InteractiveUtils` package.

In [ ]:
using InteractiveUtils: supertype
supertype(Float64)

The problem with the `supertype` function is that it does not return the whole supertype hierarchy, but only the closest *larger* supertype. For `Float64` the closest larger supertype is `AbstractFloat`. However, as in the example above, we do not want to use this supertype, since then the function will only work for floating point numbers.

### Exercise:

Create a function `supertypes_tree` which prints the whole tree of all supertypes. If the input type `T` satisfies the following condition `T === Any`, then the function should do nothing. Use the following function declaration:

```julia
function supertypes_tree(T::Type, level::Int = 0)
    # code
end
```

The optional argument `level` sets the printing indentation level.

**Hints:**
- Use the `supertype` function in combination with recursion.
- Use the `repeat` function and string with white space `"    "` to create a proper indentation.

<details>
<summary><strong>Solution:</strong></summary>


```julia
function supertypes_tree(T::Type, level::Int = 0)
    isequal(T, Any) && return
    println(repeat("   ", level), T)
    supertypes_tree(supertype(T), level + 1)
    return
end
```

The first line checks if the given input type is `Any`. If yes, then the function returns nothing. Otherwise, the function prints the type with a proper indentation provided by `repeat("   ", level)`, i.e., four white-spaces repeated `level`-times. The third line calls the `supertypes_tree` function recursively for the supertype of the input type `T` and the level of indentation `level + 1`.

</details>

Now we can use the `supertypes_tree` function to get the whole supertype hierarchy for `Float64`.

In [ ]:
supertypes_tree(Float64)

In [ ]:
Float64 <: AbstractFloat <: Real <: Number

Similarly to the `supertype` function, there is the `subtypes` function that returns all subtypes for the given type.

In [ ]:
using InteractiveUtils: subtypes
subtypes(Number)

### Exercise:

Create a function `subtypes_tree` which prints the whole tree of all subtypes for the given type. Use the following function declaration:

```julia
function subtypes_tree(T::Type, level::Int = 0)
    # code
end
```

The optional argument `level` sets the printing indentation level.

**Hints:**
- Use the `subtypes` function in combination with recursion.
- Use the `repeat` function and string with white space `"    "` to create a proper indentation.

<details>
<summary><strong>Solution:</strong></summary>

The `subtypes_tree` function is similar to `supertypes_tree`. The only differences are that we do not need to check for the top level of `Any`, and that we need to call the vectorized version `subtypes_tree.` because `subtypes(T)` returns an array.

```julia
function subtypes_tree(T::Type, level::Int = 0)
    println(repeat("   ", level), T)
    subtypes_tree.(subtypes(T), level + 1)
    return
end
```

</details>

Now we can use the `subtypes_tree` function to get the whole subtypes hierarchy for the `Number` type.

In [ ]:
subtypes_tree(Number)

## Multiple dispatch

Now we can go back to our example with the `product` function. The problem with this function is that it is too restrictive because the product of two strings is a legitimate operation that should return their concatenation. We should define a method for strings. To use the proper type, we can use the `supertypes_tree` function for the `String` type.

In [ ]:
supertypes_tree(String)

We see that the *largest* supertype for `String` is `AbstractString`. This leads to

In [ ]:
product(x::AbstractString, y::AbstractString) = x * y
product(x, y) = throw(ArgumentError("product is defined for numbers and strings only."))

We also redefined the original definition of the `product` function to throw an appropriate error.

In [ ]:
product(1, 4.5)

In [ ]:
product("a", "b")

In [ ]:
product(:a, :b)

Sometimes, it may be complicated to guess which method is used for concrete inputs. In such a case, there is a useful macro `@which` that returns the method that is called for given arguments.

In [ ]:
using InteractiveUtils: @which
@which product(1, 4.5)

In [ ]:
@which product("a", :a)

In [ ]:
@which product("a", "b")

The previous example with the `product` function shows how methods in Julia works. However, it is a good practice to use type annotation only if we want to have a specialized function or if we want to define a function, which does different things for different types of input arguments.

In [ ]:
g(x::Real) = x + 1
g(x::String) = repeat(x, 4)

For example, the `g` function returns `x + 1` if the input `x` is a real number or repeats four times the input argument if it is a string. Otherwise, it will throw a method error.

In [ ]:
g(1.2)

In [ ]:
g("a")

In [ ]:
g(:a)

#### Note

The `product` function should be defined without the type annotation. It is a good practice not to restrict input argument types unless necessary. The reason is that, in this case, there is no benefit of using the type annotation. It is better to define the function `product_new` by:

In [ ]:
product_new(x, y) = x * y

Then we can apply this function to the same inputs as the original `product` function, and we will get the same results

In [ ]:
product(1, 4.5)

In [ ]:
product_new(1, 4.5)

In [ ]:
product("a", "b")

In [ ]:
product_new("a", "b")

with only one exception

In [ ]:
product("a", :a)

In [ ]:
product_new("a", :a)

Here we get a different error. However, the error returned by the `product_new` function is more useful because it tells us what the real problem is. We can see that it is impossible to use the `*` operator to multiply a `String` and a `Symbol`. We can decide if this is the desired behaviour, and if not, we can define a method for the `*` operator that will fix it.

### Exercise:

We define the abstract type `Student` and specific types `Master` and `Doctoral`. The latter two are defined as structures containing one and three fields, respectively.

```julia
abstract type Student end

struct Master <: Student
    salary
end

struct Doctoral <: Student
    salary
    exam_mid::Bool
    exam_english::Bool
end
```

We can check that the `subtypes_tree` works correctly on any type, including the type `Student` which we defined.

```julia
subtypes_tree(Student)
```

We create instances of two students by providing values for the struct fields.

```julia
s1 = Master(5000)
s2 = Doctoral(30000, 1, 0)
```

Write the `salary_yearly` function which computes the yearly salary for both student types. The monthly salary is computed from the base salary (which can be accessed via `s1.salary`). Monthly bonus for doctoral students is 2000 for the mid exam and 1000 for the English exam.

<details>
<summary><strong>Solution:</strong></summary>

Julia prefers to write many simple functions. We write `salary_yearly` based on the not-yet-defined `salary_monthly` function.

```julia
salary_yearly(s::Student) = 12 * salary_monthly(s)
```

We specified that the input to `salary_yearly` is any `Student`. Since `Student` is an abstract type, we can call `salary_yearly` with both `Master` and `Doctoral` student. Now we need to define the `salary_monthly` function. Since the salary is computed in different ways for both students, we write two methods.

```julia
salary_monthly(s::Master) = s.salary
salary_monthly(s::Doctoral) = s.salary + s.exam_mid * 2000 + s.exam_english * 1000
```

Both methods have the same name (they are the same function) but have different inputs. While the first one is used for `Master` students, the second one for `Doctoral` students. Now we print the salary.

```julia
println("The yearly salary is $(salary_yearly(s1)).")
println("The yearly salary is $(salary_yearly(s2)).")
```

</details>

## Method ambiguities

It is possible to define a set of function methods with no most specific method applicable to some combinations of arguments.

In [ ]:
f(x::Float64, y) = x * y
f(x, y::Float64) = x + y

Here, `f` has two methods. The first method applies if the first argument is of type `Float64`, and the second method applies if the second argument is of type `Float64`.

In [ ]:
f(2.0, 3)
f(2, 3.0)

Both methods can be used if both arguments are of type `Float64`. The problem is that neither method is more specific than the other. This results in `MethodError`.

In [ ]:
f(2.0, 3.0)

We can avoid method ambiguities by specifying an appropriate method for the intersection case.

In [ ]:
f(x::Float64, y::Float64) = x - y

Now `f` has three methods.

In [ ]:
f(2.0, 3.0)